# Market/Property Evaluation Tool for Residential Real Estate Investors

Our project is designed to provide residential real estate investors with a tool that uses calculated metrics to help narrow their search for investment properties based on a set of criteria.  These metrics can be both market (city) and property based so that investors can start by narrowing from a list of cities, then filter individual properties within that market to determine which rental houses show attractive investment ratios.

**Questions our tool will answer**
1. *"In which US market should I buy a rental property?"*
2. *"Which individual properties with in a market should I take a closer look at?"*

## Core Metrics

### We will use a set of core metrics to evaluate and filter data on a market and property level. A list of these metrics can be found below, along with their calculation and goal outcome.

**Market Level**

1. City Price to Rent Ratio: Median Home Price / Annual Median Rent (monthly rent * 12), *Goal: <15*

2. Annual Population Growth % (last 5 years)

3. Annual Job Growth % (last 5 years)

4. Quality of Life Index


**Property Level**

1. Rent Ratio: Monthly Rent Estimate / Total Cost, *Goal: >1%*

2. Cap Rate:  Net Operating Income (NOI) / Total Cost, *Goal: >7%*

3. OpEx Ratio:  Operating Expenses/Gross Income, *Goal: <70%*

4. Price vs Comparables:  (List Price - Comp Price) / Price *Goal: >10%*


## Equations and Variables

### Other Formulas

1.  Total Cost = List Price + Closing Fees
2.  NOI = Gross Income - Annual Operating Expenses (OpEx)
3.  OpEx = Repairs/Maintenance + Annual Property Tax + Vacancy Rate + Insurance Estimate + Misc Expenses

### Based on the above calculations, we will need to gather the following individual data points (variable and potential source listed below):

1.  Address/List Price (US Real Estate API)
2.  Closing Fees (Formula: List Price * 3%)
3.  Gross Rent (Formula: List Price * City Price to Rent Ratio)
4.  Annual Property Tax (US Real Estate API)
5.  Insurance Estimate (Formula: (List Price/100) * $3.50
6.  Vacancy Rate (Formula: Gross Rent * 7%)
7.  Repairs/Maintenance (Formula: Gross Rent * 10%)
8.  Misc Expenses (Formula: Gross Rent * 5%)
9.  Median Home Price (Zillow csv: "Zillow Home Value Index")
10. Annual Median Rent (Zillow csv: "Zillow Observed Rent Index")
11. Quality of Life Index (Numbeo)

## Output

The results for each individual property would show the following:

Address
List Price
Total Cost
Gross Rent
Rent Ratio
Cap Rate
OpEx Ratio
Price vs Comp






In [174]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import json

In [175]:
load_dotenv()
api_key = os.getenv("X_RAPID_API_KEY")

In [176]:

#Pull in US Real Estate API
url = "https://us-real-estate.p.rapidapi.com/for-sale"

querystring = {"offset":"200","limit":"1000","state_code":"TX","city":"Houston","sort":"newest"}

headers = {
	"X-RapidAPI-Key": api_key,
	"X-RapidAPI-Host": "us-real-estate.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

#Display API Data
data = response.json()
usre_data_hou = json.dumps(data, indent=4)

#Convert Data to a Dictionary
new_dict = json.loads(usre_data_hou)
usre_data_hou_dict = new_dict["data"]["results"]



In [182]:
#Create New Dictionary and Clean Data
new_list = []
for prop in usre_data_hou_dict:
    new_dict = {
        "property_id": prop["property_id"],
        "line": prop["location"]["address"]["line"],
        "city": prop["location"]["address"]["city"],
        "state_code": prop["location"]["address"]["state_code"],
        "postal_code": prop["location"]["address"]["postal_code"],
        "list_price": prop["list_price"],
    }        
    new_list.append(new_dict)

hou_df = pd.DataFrame(new_list)
hou_df.dropna()
hou_df.rename(columns = {'line':'address'}, inplace = True)
hou_df

,property_id,address,city,state_code,postal_code,list_price
0,9761957877,Candleshade Ln,Houston,TX,77045,145000
1,9948245432,6402 Innsbruck Meadows Ln,Houston,TX,77048,249000
2,8560090354,4110T W C Jester Blvd,Houston,TX,77018,1169000
3,8423659649,3306 Nance St,Houston,TX,77020,220000
4,8581126015,4007 Maple Heights Dr,Houston,TX,77339,260000
...,...,...,...,...,...,...
195,7056667287,7730 Avenue F,Houston,TX,77012,120000
196,7421709145,15107 Lantern Creek Ln,Houston,TX,77068,375000
197,7188371425,16511 N Green Manor Dr Lot 627,Houston,TX,77396,70000
198,7883183551,1227 Country Place Dr,Houston,TX,77079,200000
